<a href="https://colab.research.google.com/github/mscholl96/mad-recime/blob/recipe1M-parser/data/recipe1M/parser/parser.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Recipe1M parser

In [1]:
!pip install quantulum3
!pip install stemming
!pip install pattern

     |████████████████████████████████| 10.7 MB 4.5 MB/s 
     |████████████████████████████████| 101 kB 10.1 MB/s 
  Created wheel for stemming: filename=stemming-1.0.1-py3-none-any.whl size=11138 sha256=821d0f46396ca5312c6d875cc64c07cf59595a8fd3d8ef48c91ad9d1e64e138a
  Stored in directory: /root/.cache/pip/wheels/6b/e5/e2/c52ebc0a5b53fd82b00cc385e57bb1c90bd50e5f54ddbc06d1
Successfully built stemming
     |████████████████████████████████| 22.2 MB 2.0 MB/s 
     |████████████████████████████████| 87 kB 6.2 MB/s 
     |████████████████████████████████| 81 kB 8.4 MB/s 
     |████████████████████████████████| 5.6 MB 49.2 MB/s 
     |████████████████████████████████| 5.6 MB 25.7 MB/s 
     |████████████████████████████████| 419 kB 34.7 MB/s 
     |████████████████████████████████| 104 kB 65.9 MB/s 
     |████████████████████████████████| 3.6 MB 55.3 MB/s 
  Created wheel for pattern: filename=Pattern-3.6-py3-none-any.whl size=22332721 sha256=dda40007785798fea92f53c5f857f6dc96e69668026694d

In [2]:
# Add GDrive
from google.colab import drive
import sys
drive.mount('/content/drive/')
sys.path.append('/content/drive/My Drive/Datasets/Recipe1M/')

Mounted at /content/drive/


In [1]:
import pandas as pd
import re
from recipe import Recipe
import requests
from bs4 import BeautifulSoup

#FILE_DIR = '../'
FILE_DIR = '/content/drive/My Drive/Datasets/Recipe1M/'

## Recipe1M data
Recipe1M comes with various json files containing crawled recipes from the web. For our project, two of them are interesting:
* layer1.json: Contains all recipes to their full extend
  
  ![layer1](https://github.com/mscholl96/mad-recime/blob/recipe1M-parser/data/recipe1M/dataset-analysis/layer1_puml.png?raw=1)

* det_ingrs.json: Only contains recipe ID, parsed ingredients and validity flag for parsing 
  
  ![det_ingrs](https://github.com/mscholl96/mad-recime/blob/recipe1M-parser/data/recipe1M/dataset-analysis/det_ingrs_puml.png?raw=1)

In our first attempt we want to make use of the parsed ingredient list and only consider recipes, where all ingredients are marked valid. The parsed ingredients don't contain amounts, so our parser has to kind of merge content of both files. Extracting ingredients from one and their amount and unit from the other file.

## Preprocessing
Removal of all invalid sets from ingredient and full data json to reduce memory. Use pickle instead of json.

In [2]:
ingredient_data = None
recipe_data = None
ingredient_file = FILE_DIR + 'det_ingrs.json'
layer1_file = FILE_DIR + 'layer1.json'

layer1_out = FILE_DIR + '2022_02_11/layer1_valid.pkl'

# Get data, set id as used index and drop unnecessary information
ingredient_data = pd.read_json(ingredient_file).set_index('id')
recipe_data = pd.read_json(layer1_file).drop(columns=['partition']).set_index('id')

# Drop recipes with more than 20 ingredients
indices = ingredient_data[[True if len(row) > 20 else False for row in ingredient_data['valid']]].index
ingredient_data = ingredient_data.drop(indices)
recipe_data = recipe_data.drop(indices)
print(f'Removed {len(indices)} recipes with more than 20 ingredients')

# Drop recipes with more than 30 instructions
indices = recipe_data[[True if len(row) > 30 else False for row in recipe_data['instructions']]].index
ingredient_data = ingredient_data.drop(indices)
recipe_data = recipe_data.drop(indices)
print(f'Removed {len(indices)} recipes with more than 30 instructions')

# Removal of all elements in ingredient json which contain invalid entries according to the data set
# Get indices of ingredients which contain false valid flags 
indices = ingredient_data[[True if any(x == False for x in row) else False for row in ingredient_data['valid']]].index
ingredient_data = ingredient_data.drop(indices).drop(columns=['valid'])
recipe_data = recipe_data.drop(indices)
print(f'Removed {len(indices)} recipes with invalid ingredients')

# Bring both datasets together
recipe_data['ingredients_parsed'] = ingredient_data['ingredients']

indices = []

# Replace fractions (such as "1/2")
fractionRegex = re.compile('d+/d+')

# Replace mixed fractions
mixedFractionRegex = re.compile('\d+(?:\s|\s?\-\s?)\d+/\d+')

# Needed to extract numbers
numberRegex = re.compile('\d+')

j = 0
for idx, recipe in recipe_data.iterrows():
  
    for ingredient in recipe['ingredients']:
        text = ingredient['text']
        if text:
            # First all mixed fractions
            mixedFractions = re.findall(mixedFractionRegex, text)
            for mixedFraction in mixedFractions:
                numbers = [int(s) for s in re.findall(numberRegex, mixedFraction)]
                float_representation = round(numbers[0] + numbers[1]/numbers[2], 2)
                text = text.replace(mixedFraction, str(float_representation))
            # Now the remaining fractions
            fractions = re.findall(fractionRegex, text)
            for fraction in fractions:
                numbers = fraction.split('/')
                float_representation = round(int(numbers[0])/int(numbers[1]), 2)
                text = text.replace(fraction, str(float_representation))
            
            # Replace original text
            ingredient['text'] = text

        elif idx not in indices:
            # To be removed later (empty ingredient)
            indices.append(idx)
    j+=1
    if j % 100000 == 0:
        print(f'Progress: {j}')


# Remove empty ingredients from data
for index in indices:
    recipe_data.loc[index]['ingredients'] = [elem for elem in recipe_data.loc[index]['ingredients'] if elem['text']]
    recipe_data.loc[index]['ingredients_parsed'] = [elem for elem in recipe_data.loc[index]['ingredients_parsed'] if elem['text']]

# Save data to pickle (it's faster)
recipe_data.drop(columns=['url']).to_pickle(layer1_out)
recipe_data


Removed 0 recipes with more than 20 ingredients
Removed 1 recipes with more than 30 instructions
Removed 6 recipes with invalid ingredients


,ingredients,url,title,instructions,ingredients_parsed
id,,,,,
000033e39b,"[{'text': '1 c. elbow macaroni'}, {'text': '1 ...",http://cookeatshare.com/recipes/dilly-macaroni...,Dilly Macaroni Salad Recipe,[{'text': 'Cook macaroni according to package ...,"[{'text': 'elbow macaroni'}, {'text': 'America..."
000035f7ed,"[{'text': '8 tomatoes, quartered'}, {'text': '...",http://www.foodnetwork.com/recipes/gazpacho1.html,Gazpacho,[{'text': 'Add the tomatoes to a food processo...,"[{'text': 'tomatoes'}, {'text': 'kosher salt'}..."
00003a70b1,"[{'text': '2 12 cups milk'}, {'text': '1 12 cu...",http://www.food.com/recipe/crunchy-onion-potat...,Crunchy Onion Potato Bake,[{'text': 'Preheat oven to 350 degrees Fahrenh...,"[{'text': 'milk'}, {'text': 'water'}, {'text':..."
00004320bb,[{'text': '1 (3 ounce) package watermelon gela...,http://www.food.com/recipe/cool-n-easy-creamy-...,Cool 'n Easy Creamy Watermelon Pie,"[{'text': 'Dissolve Jello in boiling water.'},...","[{'text': 'watermelon gelatin'}, {'text': 'boi..."
0000631d90,"[{'text': '12 cup shredded coconut'}, {'text':...",http://www.food.com/recipe/easy-tropical-beef-...,Easy Tropical Beef Skillet,"[{'text': 'In a large skillet, toast the cocon...","[{'text': 'shredded coconut'}, {'text': 'lean ..."
000075604a,"[{'text': '2 Chicken thighs'}, {'text': '2 tsp...",https://cookpad.com/us/recipes/150100-kombu-te...,Kombu Tea Grilled Chicken Thigh,[{'text': 'Pierce the skin of the chicken with...,"[{'text': 'chicken thighs'}, {'text': 'tea'}, ..."
00007bfd16,"[{'text': '6 -8 cups fresh rhubarb, or'}, {'te...",http://www.food.com/recipe/strawberry-rhubarb-...,Strawberry Rhubarb Dump Cake,[{'text': 'Put ingredients in a buttered 9 x 1...,"[{'text': 'fresh rhubarb'}, {'text': 'frozen r..."
000095fc1d,"[{'text': '8 ounces, weight Light Fat Free Van...",http://tastykitchen.com/recipes/breakfastbrunc...,Yogurt Parfaits,[{'text': 'Layer all ingredients in a serving ...,"[{'text': 'non - fat vanilla yogurt'}, {'text'..."
0000973574,"[{'text': '2 cups flour'}, {'text': '1 tablesp...",http://www.food.com/recipe/zucchini-nut-bread-...,Zucchini Nut Bread,"[{'text': 'Sift dry ingredients.'}, {'text': '...","[{'text': 'flour'}, {'text': 'cinnamon'}, {'te..."


## Plausibilty check (postprocessing)
Steps in postprocessing are:
1. Check recipe on ingredient units that obviously don't fit to the context of recipes.
2. Check for implausible amounts and remove recipes with implausible amounts
3. Convert units to metrical system and unify (e.g. all volumes to ml, all masses to g). Only leave cup and spoons as they are.

Return None if recipe is implausible, else return the processed recipe

In [21]:
import re

numberRegex = re.compile('\d+')
floatRegex = re.compile('\d+\.\d+')

def str2num(string):
    amounts = string.split('-')
    # Only a single number
    if len(amounts) == 1:
        amount = amounts[0]
    # Range of amounts, we take the upper one (the more the better;))
    else:
        amount = amounts[1]

    if re.findall(floatRegex, amount):
        return float(amount)
    
    numbers = [int(s) for s in re.findall(numberRegex, amount)]
    if not numbers:
        return 1
    if len(numbers) == 3:
        return round(numbers[0] + numbers[1]/numbers[2], 2)
    elif len(numbers) == 2:
        return round(int(numbers[0])/int(numbers[1]), 2)
    else:
        return int(numbers[0])

assert(str2num('1/2') == 0.5)
assert(str2num('1 1/2') == 1.5)
assert(str2num('1/4 - 1/2') == 0.5)
assert(str2num('1 1/2 - 2') == 2)
assert(str2num('1 - 1 1/2') == 1.5)
assert(str2num('5') == 5)
assert(str2num('42') == 42)
assert(str2num(' ') == 1)

In [32]:
# Load scraped amounts for recipes from food.com
df_amounts = pd.read_csv(FILE_DIR + 'food_com_amounts.csv').set_index('id')
df_amounts_conv = df_amounts

for idx, amounts in df_amounts.iterrows():
    amounts_conv = []
    amount_list = amounts['amounts'].replace('[', '').replace(']', '').replace("'", "").split(',')
    for elem in amount_list:
        amounts_conv.append(str2num(elem))
    df_amounts_conv.loc[idx]['amounts'] = amounts_conv

df_amounts_conv.to_csv(FILE_DIR + 'food_com_amounts_conv.csv')
df_amounts_conv.to_pickle(FILE_DIR + 'food_com_amounts_conv.pkl')

In [6]:

allowed_units = ['cup', 'tablespoon', 'teaspoon', '', 'pound-mass', 'ounce', 'millilitre', 'gram' ,'package',
 'pinch', 'quart', 'drop' ,'pint', 'inch', 'litre', 'fluid ounce', 'kilogram', 'centimetre',
 'gallon', 'centilitre', 'decilitre', 'millimetre', 'dessertspoon', 'milligram']

allowed_amounts = {'cup': 20, 'tablespoon': 20, 'teaspoon': 20, '': 50, 'pound-mass': 80, 'ounce': 50, 'millilitre': 5000, 'gram': 5000 ,'package': 20,
 'pinch': 20, 'quart': 20, 'drop': 20 ,'pint': 20, 'inch': 50, 'litre': 10, 'fluid ounce': 100, 'kilogram': 10, 'centimetre': 300,
 'gallon': 10, 'centilitre': 1000, 'decilitre': 100, 'millimetre': 3000, 'dessertspoon': 20, 'milligram': 5000}

df_food_com_amounts = pd.read_pickle(FILE_DIR + 'food_com_amounts_conv.pkl')

def postprocess(recipe):
    try:
        amounts = df_food_com_amounts.loc[recipe.id]['amounts']
    except KeyError:
        amounts = None
    if amounts and len(amounts) != len(recipe.ingredients):
        print(f'Removed recipe {recipe.id} because the number of ingredients does not match the scraped number')
        return None

    for index, ingredient in enumerate(recipe.ingredients):
        # Replace all amounts that are allowed to be replaced (if it is a food.com recipe)
        if amounts and ingredient['replaceable']:
            ingredient['amount'] = amounts[index]

        # Some of the resulting units don't fit well to usual recipe units
        if not ingredient['unit'] in allowed_units:
            print(f'Recipe {recipe.id} is not added bc {ingredient} is filtered!')
            return None
        
        # If amount is too high, we don't consider the recipe
        if ingredient['amount'] > allowed_amounts[ingredient['unit']]:
            print(f'Recipe {recipe.id} is not added bc {ingredient} exceeds limits!')
            return None

        # Convert units
        if ingredient['unit'] == 'litre':
            ingredient['unit'] = 'millilitre'
            ingredient['amount'] = ingredient['amount'] * 1000
        elif ingredient['unit'] == 'kilogram':
            ingredient['unit'] = 'gram'
            ingredient['amount'] = ingredient['amount'] * 1000
        elif ingredient['unit'] == 'centilitre':
            ingredient['unit'] = 'millilitre'
            ingredient['amount'] = ingredient['amount'] * 10
        elif ingredient['unit'] == 'decilitre':
            ingredient['unit'] = 'millilitre'
            ingredient['amount'] = ingredient['amount'] * 100
        elif ingredient['unit'] == 'millimetre':
            ingredient['unit'] = 'centimetre'
            ingredient['amount'] = ingredient['amount'] / 10
        elif ingredient['unit'] == 'dessertspoon':
            ingredient['unit'] = 'teaspoon'
            ingredient['amount'] = ingredient['amount'] * 2
        elif ingredient['unit'] == 'milligram':
            ingredient['unit'] = 'gram'
            ingredient['amount'] = ingredient['amount'] / 1000

    return recipe

## Actual parsing

In [3]:
recipes = []
# Sort by ID, 
recipe_data = pd.read_pickle(FILE_DIR + '2022_02_11/layer1_valid.pkl')
print(f'Total number of recipes: {len(recipe_data)}')
recipe_data.head(5)


Total number of recipes: 22


,ingredients,title,instructions,ingredients_parsed
id,,,,
000033e39b,"[{'text': '1 c. elbow macaroni'}, {'text': '1 ...",Dilly Macaroni Salad Recipe,[{'text': 'Cook macaroni according to package ...,"[{'text': 'elbow macaroni'}, {'text': 'America..."
000035f7ed,"[{'text': '8 tomatoes, quartered'}, {'text': '...",Gazpacho,[{'text': 'Add the tomatoes to a food processo...,"[{'text': 'tomatoes'}, {'text': 'kosher salt'}..."
00003a70b1,"[{'text': '2 12 cups milk'}, {'text': '1 12 cu...",Crunchy Onion Potato Bake,[{'text': 'Preheat oven to 350 degrees Fahrenh...,"[{'text': 'milk'}, {'text': 'water'}, {'text':..."
00004320bb,[{'text': '1 (3 ounce) package watermelon gela...,Cool 'n Easy Creamy Watermelon Pie,"[{'text': 'Dissolve Jello in boiling water.'},...","[{'text': 'watermelon gelatin'}, {'text': 'boi..."
0000631d90,"[{'text': '12 cup shredded coconut'}, {'text':...",Easy Tropical Beef Skillet,"[{'text': 'In a large skillet, toast the cocon...","[{'text': 'shredded coconut'}, {'text': 'lean ..."


In [11]:
j = 0
fileIndex = 0
for idx, raw_recipe in recipe_data.iterrows():
    
    recipe = Recipe(idx, raw_recipe['title'])
    recipe.parse_ingredients(raw_recipe['ingredients_parsed'])
    recipe.parse_instructions(raw_recipe['instructions'])
    recipe.get_ingredient_amounts(raw_recipe['ingredients'])

    # Append postprocessed recipe
    if (postprocess(recipe)):
        recipes.append(recipe)

    j += 1
    if j % 10000 == 0:
      print(f'Progress: {j}')

    # Save every 100000 recipes
    if len(recipes) == 100000:
        # Create data frame in the end (according to Stackoverflow this is faster)                
        df = pd.DataFrame([r.to_dict() for r in recipes]).set_index('id')
        path = FILE_DIR + f'2022_02_11/recipes_valid_{fileIndex}'
        fileIndex += 1
        df.to_pickle(path + '.pkl')
        del df
        recipes = []

# Save remaining data
if len(recipes) > 0:
    df = pd.DataFrame([r.to_dict() for r in recipes]).set_index('id')
    path = FILE_DIR + f'2022_02_11/recipes_valid_{fileIndex}'
    df.to_pickle(path + '.pkl')
    df.to_json(path + '.json', orient='records')
    
df.head(20)


,title,ingredients,instructions
id,,,
000033e39b,Dilly Macaroni Salad Recipe,amount unit ingredient 0 1....,0 Cook macaroni according to package direct...
000035f7ed,Gazpacho,amount unit ingredient 0 8.0 ...,0 Add the tomatoes to a food processor with...
00003a70b1,Crunchy Onion Potato Bake,amount unit ingredient 0 2...,0 Preheat oven to 350 degrees Fah...
00004320bb,Cool 'n Easy Creamy Watermelon Pie,amount unit ingredient 0 3....,0 Dissolve Jello in boiling water. 1 ...
0000631d90,Easy Tropical Beef Skillet,amount unit ingredient 0...,"0 In a large skillet, toast the coconut ove..."
000075604a,Kombu Tea Grilled Chicken Thigh,amount unit ingredient 0 2.0 ...,0 Pierce the skin of the chicken with a for...
00007bfd16,Strawberry Rhubarb Dump Cake,amount unit ing...,0 Put ingredients in a buttered 9 x 12 x 2-...
000095fc1d,Yogurt Parfaits,amount unit ingredient 0 ...,0 Layer all ingredients in a serving dish. ...
0000973574,Zucchini Nut Bread,amount unit ingredient 0 2....,0 Sift dry ingr...
